## **1. Install Selenium**

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

UsageError: Cell magic `%%shell` not found.


In [3]:
!pip install selenium

  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 22.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 34.9 MB/s eta 0:00:00
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)


## **2. Import libraries**

In [4]:
import pandas as pd
import os
import requests
import time
import random

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## **3. Crawl poem**

In [6]:
WEBDRIVER_DELAY_TIME_INT = 10

service = Service()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.implicitly_wait(5)
wait = WebDriverWait(driver, WEBDRIVER_DELAY_TIME_INT)

In [ ]:
datasets = []
deletion_script = 'arguments[0].parentNode.removeChild(arguments[0]);'
for page_idx in tqdm(range(1, 11)):
    main_url = f'https://www.thivien.net/searchpoem.php?PoemType=16&ViewType=1&Country=2&Age[]=3&Page={page_idx}'
    driver.get(main_url)

    content_tags_xpath = '//*[@class="page-content container"]/div[2]/div/div[@class="list-item"]'
    content_tags = driver.find_elements(By.XPATH, content_tags_xpath)

100%|██████████| 10/10 [00:18<00:00,  1.89s/it]


In [11]:
datasets = []
deletion_script = 'arguments[0].parentNode.removeChild(arguments[0]);'
for page_idx in tqdm(range(1, 4)):
    main_url = f'https://www.thivien.net/searchpoem.php?PoemType=16&ViewType=1&Country=2&Age[]=3&Page={page_idx}'
    driver.get(main_url)

    content_tags_xpath = '//*[@class="page-content container"]/div[2]/div/div[@class="list-item"]'
    content_tags = driver.find_elements(By.XPATH, content_tags_xpath)
    for idx in range(len(content_tags)):
        content_tag_xpath = f'/html/body/div[4]/div[2]/div/div[{2+idx}]'
        content_title_xpath = f'/html/body/div[4]/div[2]/div/div[{2+idx}]/h4/a'
        content_tag = wait.until(EC.presence_of_element_located((By.XPATH, content_tag_xpath)))
        poem_title = wait.until(EC.presence_of_element_located((By.XPATH, content_title_xpath))).text
        poem_url = wait.until(EC.presence_of_element_located((By.XPATH, content_title_xpath))).get_attribute('href')

        try:
            driver.get(poem_url)

            poem_src_xpath = '//div[@class="small"]'
            poem_content_tag = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'poem-content')))

            try:
                poem_content_i_tag = poem_content_tag.find_element(By.TAG_NAME, 'i')
                driver.execute_script(deletion_script, poem_content_i_tag)
            except:
                pass

            try:
                poem_content_b_tag = poem_content_tag.find_element(By.TAG_NAME, 'b')
                driver.execute_script(deletion_script, poem_content_b_tag)
            except:
                pass

            poem_content = poem_content_tag.text

            try:
                poem_src_tag = wait.until(EC.presence_of_element_located((By.XPATH, poem_src_xpath)))
                poem_src = poem_src_tag.text
            except:
                poem_src = ''

            poem_info = {
                'title': poem_title,
                'content': poem_content,
                'source': poem_src,
                'url': poem_url
            }

            datasets.append(poem_info)

            driver.back()
        except Exception as e:
            print(e)
            print(poem_url)

100%|██████████| 3/3 [00:15<00:00,  5.18s/it]


In [17]:
datasets = []
deletion_script = 'arguments[0].parentNode.removeChild(arguments[0]);'
for page_idx in tqdm(range(1, 4)):
    main_url = f'https://www.thivien.net/search-poem.php?PoemType=14&Page={page_idx}'
    driver.get(main_url)

    content_tags_xpath = '//*[@class="page-content container"]/div[2]/div/div[@class="list-item"]'
    content_tags = driver.find_elements(By.XPATH, content_tags_xpath)
    for idx in range(len(content_tags)):
        content_tag_xpath = f'/html/body/div[4]/div[2]/div/div[{2+idx}]'
        content_title_xpath = f'/html/body/div[4]/div[2]/div/div[{2+idx}]/h4/a'
        content_tag = wait.until(EC.presence_of_element_located((By.XPATH, content_tag_xpath)))
        poem_title = wait.until(EC.presence_of_element_located((By.XPATH, content_title_xpath))).text
        poem_url = wait.until(EC.presence_of_element_located((By.XPATH, content_title_xpath))).get_attribute('href')

        try:
            driver.get(poem_url)

            poem_src_xpath = '//div[@class="small"]'
            poem_content_tag = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'poem-content')))

            try:
                poem_content_i_tag = poem_content_tag.find_element(By.TAG_NAME, 'i')
                driver.execute_script(deletion_script, poem_content_i_tag)
            except:
                pass

            try:
                poem_content_b_tag = poem_content_tag.find_element(By.TAG_NAME, 'b')
                driver.execute_script(deletion_script, poem_content_b_tag)
            except:
                pass

            poem_content = poem_content_tag.text

            try:
                poem_src_tag = wait.until(EC.presence_of_element_located((By.XPATH, poem_src_xpath)))
                poem_src = poem_src_tag.text
            except:
                poem_src = ''

            poem_info = {
                'title': poem_title,
                'content': poem_content,
                'source': poem_src,
                'url': poem_url
            }

            datasets.append(poem_info)

            driver.back()
        except Exception as e:
            print(e)
            print(poem_url)

 33%|███▎      | 1/3 [01:49<03:38, 109.01s/it]


TimeoutException: Message: 
Stacktrace:
#0 0x599b7b3eb75a <unknown>
#1 0x599b7ae9e4b0 <unknown>
#2 0x599b7aeef9b3 <unknown>
#3 0x599b7aeefba1 <unknown>
#4 0x599b7af3e1f4 <unknown>
#5 0x599b7af155bd <unknown>
#6 0x599b7af3b5e0 <unknown>
#7 0x599b7af15363 <unknown>
#8 0x599b7aee1d63 <unknown>
#9 0x599b7aee29c1 <unknown>
#10 0x599b7b3b0a6b <unknown>
#11 0x599b7b3b4951 <unknown>
#12 0x599b7b398b62 <unknown>
#13 0x599b7b3b54c4 <unknown>
#14 0x599b7b37d13f <unknown>
#15 0x599b7b3d96f8 <unknown>
#16 0x599b7b3d98d6 <unknown>
#17 0x599b7b3ea5a6 <unknown>
#18 0x79241649caa4 <unknown>
#19 0x792416529c3c <unknown>


In [19]:
datasets = []
deletion_script = 'arguments[0].parentNode.removeChild(arguments[0]);'

for page_idx in tqdm(range(1, 4)):
    main_url = f'https://www.thivien.net/search-poem.php?PoemType=14&Page={page_idx}'
    driver.get(main_url)
    
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "list-item")))

    content_tags = driver.find_elements(By.XPATH, '//div[@class="list-item"]')

    for content_tag in content_tags:
        try:
            # Lấy tiêu đề và URL bài thơ
            title_elem = content_tag.find_element(By.XPATH, './/h4[@class="list-item-header"]/a')
            poem_title = title_elem.text
            poem_url = title_elem.get_attribute('href')

            driver.get(poem_url)

            # Lấy nội dung bài thơ
            poem_content_tag = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'poem-content')))

            # Xoá các thẻ phụ không cần thiết (nếu có)
            for tag in ['i', 'b']:
                try:
                    tag_elem = poem_content_tag.find_element(By.TAG_NAME, tag)
                    driver.execute_script(deletion_script, tag_elem)
                except:
                    pass

            poem_content = poem_content_tag.text

            # Lấy nguồn bài thơ nếu có
            try:
                poem_src = driver.find_element(By.XPATH, '//div[@class="small"]').text
            except:
                poem_src = ''

            poem_info = {
                'title': poem_title,
                'content': poem_content,
                'source': poem_src,
                'url': poem_url
            }

            datasets.append(poem_info)
            driver.back()
        except Exception as e:
            print("❌ Error at:", poem_url)
            print("Reason:", e)


100%|██████████| 3/3 [04:54<00:00, 98.31s/it] 


In [21]:
len(datasets)

30

## **4. Save data to file**

In [ ]:
df = pd.DataFrame(datasets)
df.to_csv('poem_dataset.csv', index=True)

In [ ]:
df

,title,content,source,url
0,“Bạn xấu như chiếc bóng”,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,“Chơi thân không có nghĩa”,Chơi thân không có nghĩa\nKhông cãi nhau bao g...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,“Có thể buồn chút ít”,"Có thể buồn chút ít\nMột mình, không người yêu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...
95,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
96,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [ ]:
!zip -r poem_dataset.zip poem_dataset.csv